In [2]:
import os
import numpy as np
import pandas as pd
import librosa
import torchaudio
import torch
import matplotlib.pyplot as plt
from pydub import AudioSegment
from warnings import filterwarnings
filterwarnings('ignore')
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Conv2D
from keras.models import Model
import keras.layers as layers
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

## Загрузка данных

In [3]:
with open('data_ramas_numpy_13_216.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True)

In [4]:
data.shape

(12399, 2)

In [5]:
data[0, 1].shape

(13, 216)

In [6]:
np.random.shuffle(data)

In [8]:
X = data[:, 1]
y = data[:, 0]

In [9]:
X = list(map(lambda x: x.reshape(13, 216, 1), X))

In [10]:
y = pd.get_dummies(pd.Series(y)).values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [13]:
X_train.shape, X_test.shape

((9919, 13, 216, 1), (2480, 13, 216, 1))

## Модель

In [26]:
model = Sequential()
model.add(layers.Conv2D(256, (3, 3), activation='relu', input_shape=(13, 216, 1)))
model.add(layers.MaxPooling2D((1, 1)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(11, activation='softmax'))

In [27]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 11, 214, 256)      2560      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 11, 214, 256)      0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 9, 212, 128)       295040    
_________________________________________________________________
dropout_15 (Dropout)         (None, 9, 212, 128)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 7, 210, 128)       147584    
_________________________________________________________________
dropout_16 (Dropout)         (None, 7, 210, 128)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 5, 208, 64)      

In [30]:
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [31]:
cnnhistory = model.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
620/620 [==============================] - 366s 589ms/step - loss: 2.3136 - accuracy: 0.1760 - val_loss: 2.2675 - val_accuracy: 0.1681
Epoch 2/10
620/620 [==============================] - 360s 581ms/step - loss: 2.1274 - accuracy: 0.2272 - val_loss: 2.1610 - val_accuracy: 0.2500
Epoch 3/10
620/620 [==============================] - 361s 582ms/step - loss: 2.0419 - accuracy: 0.2778 - val_loss: 2.0991 - val_accuracy: 0.2496
Epoch 4/10
620/620 [==============================] - 361s 582ms/step - loss: 1.9140 - accuracy: 0.3216 - val_loss: 2.0878 - val_accuracy: 0.2504
Epoch 5/10
620/620 [==============================] - 360s 581ms/step - loss: 1.7508 - accuracy: 0.3915 - val_loss: 2.1710 - val_accuracy: 0.2528
Epoch 6/10
620/620 [==============================] - 369s 595ms/step - loss: 1.5230 - accuracy: 0.4770 - val_loss: 2.3532 - val_accuracy: 0.2512
Epoch 7/10
620/620 [==============================] - 367s 593ms/step - loss: 1.2741 - accuracy: 0.5763 - val_loss: 2.6280 -

## Simple model logistic regression

In [33]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=uint8)

In [34]:
from sklearn.linear_model import LogisticRegression

In [44]:
X_train.shape, X_test.shape

((9919, 13, 216, 1), (2480, 13, 216, 1))

In [43]:
X_train_log = X_train.reshape(9919, 13 * 216)

In [45]:
X_test_log = X_test.reshape(2480, 13 * 216)

In [70]:
y_train_log = np.array(list(map(lambda x: np.where(x == 1)[0][0], y_train)))

In [73]:
y_test_log = np.array(list(map(lambda x: np.where(x == 1)[0][0], y_test)))

In [71]:
clf = LogisticRegression(random_state=0, multi_class='multinomial', solver='lbfgs', penalty='l2', C=1.0)

In [72]:
clf.fit(X_train_log, y_train_log)

LogisticRegression(multi_class='multinomial', random_state=0)

In [77]:
clf.score(X_train_log, y_train_log), clf.score(X_test_log, y_test_log)

(0.3224115334207077, 0.23629032258064517)

## Simple LGBM model

In [79]:
! pip install lightgbm

In [81]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train_log, y_train_log)

LGBMClassifier()

In [82]:
y_pred_train = clf.predict(X_train_log)

In [84]:
from sklearn.metrics import accuracy_score

In [85]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train_log, y_pred_train)))

Training-set accuracy score: 0.9748


In [86]:
y_pred=clf.predict(X_test_log)

In [87]:
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test_log, y_pred)))

LightGBM Model accuracy score: 0.2669


## SVC

In [88]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [89]:
from sklearn.svm import SVC

In [90]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [93]:
clf.fit(X_train_log, y_train_log)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [94]:
clf.score(X_train_log, y_train_log)

0.47877810263131365

In [95]:
clf.score(X_test_log, y_test_log)

0.28266129032258064

## LSTM

In [99]:
X_train_log.shape

(9919, 2808)

In [97]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)

In [103]:
# create the model
embedding_vecor_length = 2808
model = Sequential()
model.add(LSTM(100, ))
model.add(Dense(11, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [104]:
model.fit(X_train_log, y_train_log, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3


ValueError: in user code:

    C:\Users\79217\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\79217\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\79217\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\79217\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\79217\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\79217